In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
import re          
import os          
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PUNAM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [12]:
data = pd.read_excel('20211030 Test Assignment/input.xlsx')

for i in tqdm(range(0, len(data))):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0'}
    url = data['URL'][i]
    r = requests.get(url, headers=headers)
    htmlcontent = r.content
    soup = BeautifulSoup(htmlcontent, 'html.parser')
    mydivs = soup.find("div", {"class": "td-post-content"})
    #Removing pre tag from mydivs
    try:
        mydivs.pre.decompose()
    except:
        pass
    #Storing news articles in articles directory
    text_file = open('articles/' + str(i + 1) + '.txt', "w", encoding="utf-8")
    text_file.write(soup.title.getText())
    text_file.write(mydivs.getText() if mydivs else "pass")
    text_file.close()


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [03:40<00:00,  1.47s/it]


In [13]:
df = pd.read_excel('Output Data Structure.xlsx')

In [18]:
#Reading all files available in the articles directory.
#Article directory consists of all 150 articles stored in the text format 
for filename1 in tqdm(os.listdir('articles')):
    f = open('articles/' + filename1, 'r', encoding="utf8") 
    text = f.read()                                         
    f.close()              
    #Converting text into lowercases
    text1 = text.lower()   
    #Tocknizing text1 with the help of nltk word_toknize method
    text2 = word_tokenize(text1)                            
    sent = sent_tokenize(text1)                             
    #Removing all the punctuation from the text
    text3 = []                            
    for i in text2:                       
        res = re.sub(r'[^ \w\s]', "", i)  
        if res != "":                     
            text3.append(res)             
    #Creating stopword list from the given stopwords from stopwords directory 
    stopwords = []                                                          
    for filename in os.listdir('20211030 Test Assignment/StopWords'):       
        f1 = open('20211030 Test Assignment/StopWords/' + filename, 'r')    
        for i in f1:                                                        
         # print(i)                                                      
            if '|' in i:                                                    
                x = re.split("\s", i)                                       
                x.remove("")                                                
                stopwords.append(x[0])                                      
            else:                                                           
                stopwords.append(i.strip())                                 
        f1.close()                                                          
    #Excluding all the stopwords form the text
    text4 = []                
    for i in text3:           
        if i not in stopwords:
            text4.append(i)   

    # Reading -ve and +ve words from txt file                                       
    f2 = open('20211030 Test Assignment/MasterDictionary/negative-words.txt', 'r')  
    nve =  f2.read().split()                                                                                                         
    f2.close()                                                                      
    f2 = open('20211030 Test Assignment/MasterDictionary/positive-words.txt', 'r')  
    pve = f2.read().split()                                                        
    f2.close()                                                                      

    # Counting negative and positive words news article  
    positiveScore, negativeScore = 0, 0                  
    for i in text4:                                      
        if i in pve:                                     
            positiveScore += 1                           
        elif i in nve:                                   
            negativeScore += 1                           
    #Calculating polarity Score
    polarityScore = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    #Calculating subjectivity Score
    subjectivityScore = (positiveScore + negativeScore) / (len(text4) + 0.000001)
    #Calculating average Sentence Length
    averageSentenceLength = len(text3) / len(sent)
    #Counting complex words
    complexwords = 0                                                      
    for i in text4:                                                       
        s = 0                                                             
        for j in i:                                                       
            if (j == 'a' or j == 'e' or j == 'i' or j == 'o' or j == 'u'):
                s += 1                                                    
        if s > 2:                                                         
            complexwords = complexwords + 1                     
    #Calculating percentage of Complex words
    percentageofComplexwords = complexwords / len(text4)
    #Calculating fog Index
    fogIndex = 0.4 * (averageSentenceLength + percentageofComplexwords)
    #Calculating average Number of Words PerSentence
    averageNumberofWordsPerSentence = len(text4) / len(sent)
    
    fname = int(filename1.removesuffix('.txt')) - 1
    #Counting personal pronouns used in article
    personal_noun =['I', 'we','my','ours', 'us','My','We','Ours','Us',] 
    personalPronouns=0
    for word in text3:
        if word in personal_noun:
            personalPronouns+=1
    
    #Calculating average Word Length
    letterCount = 0                             
    for i in text4:                             
        for j in i:                             
            letterCount += 1                    
    averageWordLength = letterCount / len(text4)
   
    #Counting total syllabul present in article
    syllable = 0
    for i in text4:
        s = 0
        for j in i:
            if (j == 'a' or j == 'e' or j == 'i' or j == 'o' or j == 'u'):
                s += 1
        if i.endswith("es") or i.endswith("ed"):
            s -= 1
        if s > 0:
            syllable +=1
    
    #Storing all the computed data in their respective column
    df.at[fname,'POSITIVE SCORE'] = positiveScore
    df.at[fname,'NEGATIVE SCORE'] = negativeScore
    df.at[fname,'POLARITY SCORE'] = polarityScore
    df.at[fname,'SUBJECTIVITY SCORE'] = subjectivityScore
    df.at[fname,'AVG SENTENCE LENGTH'] = averageSentenceLength
    df.at[fname,'PERCENTAGE OF COMPLEX WORDS'] = percentageofComplexwords
    df.at[fname,'FOG INDEX'] = fogIndex
    df.at[fname,'AVG NUMBER OF WORDS PER SENTENCE'] = averageNumberofWordsPerSentence 
    df.at[fname,'COMPLEX WORD COUNT'] = complexwords
    df.at[fname,'WORD COUNT'] = len(text4)
    df.at[fname,'SYLLABLE PER WORD'] = syllable/len(text4)
    df.at[fname,'PERSONAL PRONOUNS'] = personalPronouns
    df.at[fname,'AVG WORD LENGTH'] = averageWordLength

100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:15<00:00,  9.38it/s]


In [19]:
df = df.astype({'URL_ID':int})
df.to_csv('Output Data Structure.csv',index=False)